In [24]:
import numpy as np
import json
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import multivariate_normal
from scipy.stats import invwishart as iw
from data_fusion import Data_fusion
import matplotlib.pyplot as plt
from dict_search import gen_dict_extract
import numpy.linalg as la
from data_structure import Data

techniques = ["bay", "ei", "ci", "pc"]

fusions = {"bay": Data_fusion(0), "ei": Data_fusion(1), "ci": Data_fusion(2), "pc": Data_fusion(4)}

def calculate_distance(mse, anees):
    return np.sqrt((mse)**2 + (anees-3)**2)


zero = np.zeros((3,1))
df = 10

emitter_n = 6

def inv(M):
    if M.size == 1:
        return M**(-1)
    else:
        return np.linalg.inv(M)
    

def normalize(v):
    return v * (np.linalg.norm(v))**(-1)

def MSE(u, v):
    return  ((u - v)**2).mean(axis=None)

In [25]:
def plot_data_inputs(robot_locs, satellite_locs):
    pass
#     fig = plt.figure()
#     ax = plt.axes(projection='3d')
#     ax.scatter(robot_locs[:,0], robot_locs[:,1], robot_locs[:,2])
#     ax.scatter(satellite_locs[:,0], satellite_locs[:,1], satellite_locs[:,2])

# Data Generation

###### Reads in "node_config.json" and computes H vectors (vectors that connect emitter and observer) 

In [26]:
def get_H(robot_num, emitters):
    robot_loc = np.array(json.load(open("robot_poses.json"))["positions"])
    n = 0
    
    H = np.zeros((len(emitters), 3))
    i = 0
    for node in emitters:
        H[i,:][:3] = np.array(node) - robot_loc[robot_num]
        H[i,:][:3] /= la.norm(H[i,:][:3])
        i+=1

    
    plot_data_inputs(robot_loc, np.array(emitters))
    
    return H

def get_emitters():
    fin = open("node_config.json") 
    inp = json.load(fin)

    emitters = np.array([inp['emitters']['GROUND']['data'], 
                        inp['emitters']['GPS']['data'],
                        inp['emitters']['GLONASS']['data'],
                        inp['emitters']['GALILEO']['data']]).reshape(4*len(inp['emitters']['GROUND']['data']), 3)
#     emitters = np.array([inp['emitters']['GPS']['data'],
#                         inp['emitters']['GLONASS']['data'],
#                         inp['emitters']['GALILEO']['data']]).reshape(3*len(inp['emitters']['GPS']['data']), 3)

    fin.close()
    return emitters


def get_corr():
    fin = open("node_config.json") 
    inp = json.load(fin)
    corr_s = [inp['emitters']['GROUND']['corr'], 
              inp['emitters']['GPS']['corr'], 
              inp['emitters']['GLONASS']['corr'], 
              inp['emitters']['GALILEO']['corr']]
#     corr_s = [inp['emitters']['GPS']['corr'], 
#               inp['emitters']['GLONASS']['corr'], 
#               inp['emitters']['GALILEO']['corr']]

    fin.close()
    
    return corr_s, 4*len(inp['emitters']['GPS']['data'])


def generate_R(corr, n):
    diag = np.random.rand(n)
    R = np.diag(diag)
    for i in range(n):
        for j in range(i):
            R[i][j] = np.sqrt(diag[i])*np.sqrt(diag[j])*corr[int(i/emitter_n)]
            R[j][i] = R[i, j]
    return R

###### Returns projected velocity along corresponding geometric vector for emitter (H--> geometric vector)

In [27]:
def get_true_Y(H):
    true_mu = np.random.multivariate_normal(zero.reshape(3,), 50*np.identity(3)).reshape(3,1)
    return (H@true_mu).reshape(H.shape[0], 1), true_mu

###### Returns "observed" y value that adds error to the true y based on the randomly generated R variance

In [28]:
def get_y(true_Y, R):
    Y = np.zeros((n , 1))
    for i in range(true_Y.size):
        Y[i] = np.random.multivariate_normal(true_Y[i], np.array([[R[i]]]))
#         Y[i] = true_Y[i]
    return Y

## Fusion Integration

In [29]:
def getInitialInformationMatrix(data):
    I = np.zeros((3, 3))
    i = np.zeros((3, 1))
    for _ in range(10):
        H, y, R = data.getNextBatch()
        I = I + H.T @ H * inv(R) 
        i = i + H.T * inv(R) * y
    return I, i

    
    
def add_satellite(data, I, i, fusion):
    H, y, R = data.getNextBatch()

    #compute state dimensional from information 
    C = np.linalg.inv(I)
    mu = C @ i
    
    #normalize H first
    u = H
    
    #project mat/vec in direction of u
    mu_a = u @ mu
    C_a = u @ C @ u.T

    #set-up b distribution
    mu_b = np.array(y).reshape(1, 1)
    C_b = np.array(R).reshape(1, 1)
    
    
    x_f, C_f = fusion.fuse(mu_a, mu_b, C_a, C_b)

    
    #Compute additional information
    D = inv(inv(C_f) - inv(C_a))
    x_d = D @ (inv(C_f) @ x_f - inv(C_a) @ mu_a)
    
    
    fused_I = I + inv(D) * u.T @ u
    fused_i = i + u.T * inv(D) * x_d
    return fused_I, fused_i


In [30]:

results = {
            "mse":{
                "bay":[], 
                "ei":[], 
                "ci": [], 
                "pc": []
                }, 
            "anees":{
                "bay":[], 
                "ei":[], 
                "ci": [], 
                "pc": []
            }
}

corr, n = get_corr()
emitters = get_emitters()

R = generate_R(corr, n)
print(R)


[[4.22786770e-01 1.00695654e+00 9.83488992e-01 7.24833280e-01
  8.05871491e-01 5.04122638e-01 8.71727796e-01 4.78878144e-01
  7.18259309e-01 5.43938951e-01 6.71978566e-01 4.61141203e-01
  5.08592298e-01 1.81441890e-01 6.97735802e-01 4.85726220e-01
  6.15143799e-01 7.58361532e-01 1.01814889e-03 5.96293850e-02
  6.49739935e-02 5.99210217e-02 8.14281593e-02 8.86164670e-02]
 [1.00695654e+00 9.23569930e-01 1.45359594e+00 1.07130301e+00
  1.19107742e+00 7.45092853e-01 1.28841298e+00 7.07781512e-01
  1.06158668e+00 8.03941333e-01 9.93183781e-01 6.81566328e-01
  7.51699008e-01 2.68170968e-01 1.03125296e+00 7.17902964e-01
  9.09182041e-01 1.12085774e+00 1.50482323e-03 8.81321831e-02
  9.60315102e-02 8.85632219e-02 1.20350754e-01 1.30975067e-01]
 [9.83488992e-01 1.45359594e+00 8.81023191e-01 1.04633584e+00
  1.16331886e+00 7.27728152e-01 1.25838598e+00 6.91286367e-01
  1.03684596e+00 7.85205144e-01 9.70037217e-01 6.65682139e-01
  7.34180348e-01 2.61921132e-01 1.00721917e+00 7.01171934e-01
  8.87

In [31]:
for i in range(500):
    H= get_H(i, emitters)
    true_Y, mu = get_true_Y(H)

    Y = get_y(true_Y, np.diag(R))

    data = Data(Y, H, np.diag(R))

    for tech in techniques:
        data.reset()
        info_mat, info_vec = getInitialInformationMatrix(data)
        while not data.completed():
            info_mat, info_vec = add_satellite(data, info_mat, info_vec, fusions[tech])
            
        pred = ((inv(info_mat) @ info_vec))
        covs = np.sqrt(np.diag(inv(info_mat)))[:3].reshape(3,)
        error = (pred - mu).reshape(3,)

        results["mse"][tech].append(MSE(mu, pred))
        P_inv = inv(inv(info_mat)[:3,:3])
        results["anees"][tech].append(error @ P_inv @ error)

In [32]:
print("tech"+ '\t|\t' + "MSE" + '\t\t\t|\t' + "ANEES" + '\t\t\t|\t' + "DISTANCE")
for tech in techniques:
    mse = np.average(results["mse"][tech])
    anees = np.average(results["anees"][tech])
    distance = calculate_distance(mse, anees)
    print(tech + ':\t|\t' + str(mse) + '\t|\t' + str(anees) + '\t|\t' + str(distance))

tech	|	MSE			|	ANEES			|	DISTANCE
bay:	|	0.1755489008430143	|	8527.01725327019	|	8524.017255077873
ei:	|	0.18224300004269484	|	8525.568868296154	|	8522.568870244657
ci:	|	0.298601224551695	|	4258.449324630009	|	4255.449335106305
pc:	|	0.1784271052359386	|	8526.167744721119	|	8523.167746588748


In [43]:
print(results["anees"]["bay"].index(max(results["anees"]["bay"])))
print(max(results["anees"]["bay"]))
print(results["anees"]["ei"].index(max(results["anees"]["ei"])))
print(max(results["anees"]["ei"]))
print(results["anees"]["ci"].index(max(results["anees"]["ci"])))
print(max(results["anees"]["ci"]))
print(results["anees"]["pc"].index(max(results["anees"]["pc"])))
print(max(results["anees"]["pc"]))


70
17.606880815728708
70
17.456785078403687
23
13.336267832426953
70
16.682904586608817



In [45]:
# results["anees"]["bay"].remove(max(results["anees"]["bay"]))
results["anees"]["ci"].remove(max(results["anees"]["ci"]))
results["anees"]["ei"].remove(max(results["anees"]["ei"]))
results["anees"]["pc"].remove(max(results["anees"]["pc"]))
print("tech"+ '\t|\t' + "MSE" + '\t\t\t|\t' + "ANEES")
for tech in techniques:
    print(tech + ':\t|\t' + str(np.average(results["mse"][tech])) + '\t|\t' + str(np.average(results["anees"][tech])))

tech	|	MSE			|	ANEES
bay:	|	0.1961894993469102	|	5.280783000160023
ei:	|	0.22725646901193158	|	4.001529720769044
ci:	|	0.26487062454359533	|	3.2477021391984118
pc:	|	0.19781633482024463	|	4.066371779247262
